In [1]:
import sys
sys.path.insert(1, '../unseen')
import pdb

import dask.array as da
from dask.distributed import Client, LocalCluster
import xclim.analog

import fileio
import bootstrap

In [2]:
cluster = LocalCluster(n_workers=1, threads_per_worker=1)
client = Client(cluster)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:40665 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 44.92 GiB


## Read data

In [4]:
cafe_file = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_19900501-20201101_A-DEC-mean_wheat-sheep-mean_bias-corrected-awap-additive.zarr.zip'
ds_cafe = fileio.open_file(cafe_file)
ds_cafe          

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 62, lead_time: 9, region: 4)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1991-05-01 00:00:00 ... 2021-11-01 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8
  * region     (region) <U10 'south-west' 'south-east' 'north-east' 'all'
    time       (lead_time, init_date) object dask.array<chunksize=(9, 62), meta=np.ndarray>
Data variables:
    pr         (init_date, lead_time, ensemble, region) float64 dask.array<chunksize=(1, 9, 96, 4), meta=np.ndarray>
Attributes:
    history:  Mon Aug 09 14:20:03 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [5]:
awap_file = '/g/data/xv83/dbi599/pr_awap_1900-2020_A-DEC-mean_wheat-sheep-mean.zarr.zip'
ds_awap = fileio.open_file(awap_file, sel={'time': slice('1990-01-01', '2020-12-31')})
ds_awap

<xarray.Dataset>
Dimensions:  (region: 4, time: 31)
Coordinates:
  * region   (region) <U10 'south-west' 'south-east' 'north-east' 'all'
  * time     (time) object 1990-01-01 09:00:00 ... 2020-01-02 09:00:00
Data variables:
    pr       (time, region) float64 dask.array<chunksize=(1, 4), meta=np.ndarray>
Attributes:
    history:  Mon Aug 09 14:19:09 2021: /g/data/e14/dbi599/miniconda3/envs/un...

## K-S test

In [6]:
def run_dask_tasks(ds, var, chunk_dim=None):
    """Run the tasks in an xarray data array"""
    
    dims = ds['pr'].dims
    values = ds['pr'].compute().values

    chunks = list(values.shape)
    if chunk_dim:
        index = dims.index(chunk_dim)
        chunks[index] = 1
    
    ds.update({var: (dims, da.from_array(values, chunks=chunks))})
               
    return ds

In [7]:
fcst_stacked = ds_cafe.stack({'sample': ['ensemble', 'init_date', 'lead_time']})
fcst_stacked = fcst_stacked.chunk({'sample': -1, 'region': 1})

In [8]:
fcst_stacked['pr']

<xarray.DataArray 'pr' (region: 4, sample: 53568)>
dask.array<rechunk-merge, shape=(4, 53568), dtype=float64, chunksize=(1, 53568), chunktype=numpy.ndarray>
Coordinates:
  * region     (region) <U10 'south-west' 'south-east' 'north-east' 'all'
    time       (sample) object dask.array<chunksize=(53568,), meta=np.ndarray>
  * sample     (sample) MultiIndex
  - ensemble   (sample) int64 1 1 1 1 1 1 1 1 1 1 ... 96 96 96 96 96 96 96 96 96
  - init_date  (sample) object 1991-05-01 00:00:00 ... 2021-11-01 00:00:00
  - lead_time  (sample) int64 0 1 2 3 4 5 6 7 8 0 1 2 ... 7 8 0 1 2 3 4 5 6 7 8
Attributes:
    bias_correction_method:  additive
    bias_correction_period:  1990-01-01-2020-12-31
    cell_methods:            time: mean
    interp_method:           conserve_order1
    long_name:               Total precipitation rate
    time_avg_info:           average_T1,average_T2,average_DT
    units:                   mm d-1

In [9]:
fcst_stacked = run_dask_tasks(fcst_stacked, 'pr', chunk_dim='region')
fcst_stacked['pr']

<xarray.DataArray 'pr' (region: 4, sample: 53568)>
dask.array<array, shape=(4, 53568), dtype=float64, chunksize=(1, 53568), chunktype=numpy.ndarray>
Coordinates:
  * region     (region) <U10 'south-west' 'south-east' 'north-east' 'all'
    time       (sample) object dask.array<chunksize=(53568,), meta=np.ndarray>
  * sample     (sample) MultiIndex
  - ensemble   (sample) int64 1 1 1 1 1 1 1 1 1 1 ... 96 96 96 96 96 96 96 96 96
  - init_date  (sample) object 1991-05-01 00:00:00 ... 2021-11-01 00:00:00
  - lead_time  (sample) int64 0 1 2 3 4 5 6 7 8 0 1 2 ... 7 8 0 1 2 3 4 5 6 7 8

In [11]:
awap_faux_stacked = ds_awap.rename(time='sample')
#awap_faux_stacked['sample'] = np.arange(awap_faux_stacked['sample'].size)
awap_faux_stacked = awap_faux_stacked.chunk({'sample': -1, 'region': 1})
awap_faux_stacked['pr']

<xarray.DataArray 'pr' (sample: 31, region: 4)>
dask.array<rechunk-merge, shape=(31, 4), dtype=float64, chunksize=(31, 1), chunktype=numpy.ndarray>
Coordinates:
  * region   (region) <U10 'south-west' 'south-east' 'north-east' 'all'
  * sample   (sample) object 1990-01-01 09:00:00 ... 2020-01-02 09:00:00
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm d-1
    valid_range:                   [-1.0, 100000.0]

In [12]:
awap_faux_stacked = run_dask_tasks(awap_faux_stacked, 'pr', chunk_dim='region')
awap_faux_stacked['pr']

<xarray.DataArray 'pr' (sample: 31, region: 4)>
dask.array<array, shape=(31, 4), dtype=float64, chunksize=(31, 1), chunktype=numpy.ndarray>
Coordinates:
  * region   (region) <U10 'south-west' 'south-east' 'north-east' 'all'
  * sample   (sample) object 1990-01-01 09:00:00 ... 2020-01-02 09:00:00

In [14]:
xclim.analog.spatial_analogs(target=awap_faux_stacked.compute(),
                             candidates=fcst_stacked.compute(),
                             dist_dim='sample',
                             method='kolmogorov_smirnov')

<xarray.DataArray 'dissimilarity' (region: 4)>
array([0.16674134, 0.16875747, 0.17769937, 0.19500448])
Coordinates:
  * region   (region) <U10 'south-west' 'south-east' 'north-east' 'all'
Attributes:
    long_name:  Dissimilarity between target and candidates, using metric kol...
    indices:    pr
    metric:     kolmogorov_smirnov

In [15]:
ks_obs = xclim.analog.spatial_analogs(target=awap_faux_stacked,
                                      candidates=fcst_stacked,
                                      dist_dim='sample',
                                      method='kolmogorov_smirnov')

In [16]:
ks_obs

<xarray.DataArray 'dissimilarity' (region: 4)>
dask.array<transpose, shape=(4,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * region   (region) <U10 'south-west' 'south-east' 'north-east' 'all'
Attributes:
    long_name:  Dissimilarity between target and candidates, using metric kol...
    indices:    pr
    metric:     kolmogorov_smirnov

In [ ]:
ks_obs.values

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


For a 2D KS-test I think you just include two variables in the Dataset instead of just one. For instance, at the moment `data_vars` only has one variable, but you could add another.

In [11]:
fcst_stacked.data_vars

Data variables:
    pr       (sample) float64 dask.array<chunksize=(8448,), meta=np.ndarray>

# Resampling (bootstrapping)

Need to test random sub-samples from the forecast data (with same length as the observations) against the entire forecast sample to get the distribution of values you might expect if the observations come from the same population as the forecast. 

In [12]:
n_samples = len(awap_faux_stacked['sample'])
n_samples

31

In [13]:
samples = {'sample': (n_samples, 1)}

### Single random sample

In [14]:
fcst_random_sample = bootstrap.random_resample(fcst_stacked, samples=samples)
fcst_random_sample

<xarray.Dataset>
Dimensions:    (region: 4, sample: 31)
Coordinates:
  * region     (region) <U10 'south-west' 'south-east' 'north-east' 'all'
    time       (sample) object dask.array<chunksize=(31,), meta=np.ndarray>
  * sample     (sample) MultiIndex
  - ensemble   (sample) int64 45 85 6 16 41 85 96 86 ... 32 13 54 2 13 52 75 41
  - init_date  (sample) object 1992-05-01 00:00:00 ... 2011-11-01 00:00:00
  - lead_time  (sample) int64 2 3 0 4 4 1 8 4 0 1 0 5 ... 3 0 5 3 8 3 1 5 1 0 7
Data variables:
    pr         (region, sample) float64 dask.array<chunksize=(1, 31), meta=np.ndarray>
Attributes:
    history:  Mon Aug 09 14:20:03 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [15]:
ks_random = xclim.analog.spatial_analogs(target=fcst_random_sample,
                                         candidates=fcst_stacked,
                                         dist_dim='sample',
                                         method='kolmogorov_smirnov')

In [16]:
ks_random

<xarray.DataArray 'dissimilarity' (region: 4)>
dask.array<transpose, shape=(4,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * region   (region) <U10 'south-west' 'south-east' 'north-east' 'all'
Attributes:
    long_name:  Dissimilarity between target and candidates, using metric kol...
    indices:    pr
    metric:     kolmogorov_smirnov

In [17]:
ks_random.values

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


KilledWorker: ("('vectorize_kolmogorov_smirnov-vectorize_kolmogorov_smirnov_0-transpose-75bfca75d494758ee7913d2b4c0fb5b2', 1)", <WorkerState 'tcp://127.0.0.1:40117', name: 1, memory: 0, processing: 4>)

You get the same result no matter which array is the target and which is the candidate.

### n random samples

In [18]:
n_repeats = 50
function = xclim.analog.spatial_analogs
function_kwargs = {'candidates': fcst_stacked,
                   'dist_dim': 'sample',
                   'method': 'kolmogorov_smirnov'}

dist = bootstrap.n_random_resamples(fcst_stacked,
                                    samples=samples,
                                    n_repeats=n_repeats,
                                    function=function,
                                    function_kwargs=function_kwargs,
                                    bundle_args=False,
                                    with_dask=True)

In [19]:
dist

<xarray.DataArray 'dissimilarity' (k: 50)>
array([0.06837552, 0.08134254, 0.0462401 , 0.07318569, 0.06663223,
       0.08479683, 0.07848011, 0.12346117, 0.09476154, 0.09722581,
       0.09499828, 0.07804967, 0.05447228, 0.07940556, 0.07682292,
       0.07524105, 0.0450241 , 0.11463714, 0.04173123, 0.09142562,
       0.07025869, 0.05686123, 0.07399277, 0.07419723, 0.08959625,
       0.04144069, 0.12907843, 0.08983299, 0.05213714, 0.09237259,
       0.06121944, 0.05544077, 0.09416968, 0.09112431, 0.04522856,
       0.04303332, 0.08517347, 0.08840177, 0.05968061, 0.0591318 ,
       0.13778409, 0.07919034, 0.06659995, 0.07503659, 0.10455406,
       0.09626808, 0.06551309, 0.09708592, 0.09046789, 0.06778366])
Dimensions without coordinates: k
Attributes:
    long_name:  Dissimilarity between target and candidates, using metric kol...
    indices:    pr
    metric:     kolmogorov_smirnov

In [20]:
dist.values

array([0.06837552, 0.08134254, 0.0462401 , 0.07318569, 0.06663223,
       0.08479683, 0.07848011, 0.12346117, 0.09476154, 0.09722581,
       0.09499828, 0.07804967, 0.05447228, 0.07940556, 0.07682292,
       0.07524105, 0.0450241 , 0.11463714, 0.04173123, 0.09142562,
       0.07025869, 0.05686123, 0.07399277, 0.07419723, 0.08959625,
       0.04144069, 0.12907843, 0.08983299, 0.05213714, 0.09237259,
       0.06121944, 0.05544077, 0.09416968, 0.09112431, 0.04522856,
       0.04303332, 0.08517347, 0.08840177, 0.05968061, 0.0591318 ,
       0.13778409, 0.07919034, 0.06659995, 0.07503659, 0.10455406,
       0.09626808, 0.06551309, 0.09708592, 0.09046789, 0.06778366])